<a href="https://colab.research.google.com/github/Hexclan14/AutoMediQu/blob/main/AutoMediQu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import time
import random
import threading
import websocket
import json

# --- Simulated Data Capture (for POC) ---

def simulate_pen_strokes(text="Hello", delay=0.1):
    """Simulates pen stroke data for a given text."""
    strokes = []
    for char in text:
        char_strokes = []
        for _ in range(random.randint(5, 20)):
            x = random.randint(0, 100)
            y = random.randint(0, 100)
            t = time.time()
            char_strokes.append((x, y, t))
        strokes.append(char_strokes)
        time.sleep(delay)  # Simulate time between characters.
    return strokes

# --- Real-World Data Capture (Conceptual) ---

class RealTimePenCapture:
    def __init__(self, websocket_url):
        self.websocket_url = websocket_url
        self.strokes = []
        self.ws = None
        self.running = False

    def on_message(self, ws, message):
        try:
            data = json.loads(message)
            if "stroke_data" in data:
                self.strokes.append(data["stroke_data"])
        except json.JSONDecodeError:
            print("Invalid JSON received.")

    def on_error(self, ws, error):
        print(f"WebSocket error: {error}")

    def on_close(self, ws, close_status_code, close_msg):
        print("WebSocket connection closed.")
        self.running = False

    def on_open(self, ws):
        print("WebSocket connection opened.")

    def start_capture(self):
        self.running = True
        self.ws = websocket.WebSocketApp(
            self.websocket_url,
            on_message=self.on_message,
            on_error=self.on_error,
            on_close=self.on_close,
            on_open=self.on_open,
        )

        def run_forever():
            self.ws.run_forever()

        self.thread = threading.Thread(target=run_forever)
        self.thread.daemon = True  # Allow the main thread to exit
        self.thread.start()

    def stop_capture(self):
        if self.ws:
            self.ws.close()
            self.running = False
            self.thread.join()

    def get_strokes(self):
        return self.strokes
# Example real world usage.
# real_capture = RealTimePenCapture("ws://your-pen-device-websocket-url")
# real_capture.start_capture()
# time.sleep(10) #capture data for 10 seconds.
# real_capture.stop_capture()
# captured_data = real_capture.get_strokes()
# print(captured_data)

#Example simulation usage.
if __name__ == "__main__":
    simulated_data = simulate_pen_strokes("Test")
    print("Simulated data: ", simulated_data)

Simulated data:  [[(58, 26, 1740551062.4530122), (56, 17, 1740551062.4530153), (99, 100, 1740551062.4530177), (1, 31, 1740551062.453025), (74, 88, 1740551062.4530268), (78, 12, 1740551062.4530287), (66, 55, 1740551062.4530299), (16, 48, 1740551062.453034), (28, 29, 1740551062.4530358), (33, 23, 1740551062.4530373), (66, 93, 1740551062.4530387)], [(15, 16, 1740551062.55312), (54, 80, 1740551062.553123), (9, 27, 1740551062.5531244), (55, 76, 1740551062.5531259), (70, 95, 1740551062.5531278), (51, 20, 1740551062.55313), (38, 82, 1740551062.5531313), (46, 98, 1740551062.5531325), (31, 18, 1740551062.5531337), (9, 96, 1740551062.5531359), (27, 18, 1740551062.553137), (55, 96, 1740551062.5531383), (60, 44, 1740551062.5531397), (77, 97, 1740551062.5531409), (17, 38, 1740551062.553142), (12, 94, 1740551062.5531433), (98, 45, 1740551062.5531447)], [(59, 81, 1740551062.6532712), (17, 56, 1740551062.6532748), (90, 9, 1740551062.653277), (6, 11, 1740551062.6532784), (46, 38, 1740551062.6532805), (

In [4]:
from google.colab import files

In [5]:
files.upload("kaggle.json")

Saving kaggle (1).json to kaggle.json/kaggle (1).json


{'kaggle.json/kaggle (1).json': b'{"username":"hexclan14","key":"10088327f103878be849d9ccb3207284"}'}

In [6]:
import os


!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json  # Set permissions

!kaggle datasets download -d mamun1113/doctors-handwritten-prescription-bd-dataset

!unzip doctors-handwritten-prescription-bd-dataset.zip -d doctors_dataset


Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 4, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.11/dist-packages/kaggle/__init__.py", line 7, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 414, in authenticate
    self._load_config(config_data)
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 465, in _load_config
    raise ValueError('Error: Missing %s in configuration.' % item)
ValueError: Error: Missing username in configuration.
unzip:  cannot find or open doctors-handwritten-prescription-bd-dataset.zip, doctors-handwritten-prescription-bd-dataset.zip.zip or doctors-handwritten-prescription-bd-dataset.zip.ZIP.


In [2]:
def preprocess_strokes(strokes):
    """Preprocesses the simulated pen stroke data."""
    processed_data = []
    for char_strokes in strokes:
        processed_data.append(char_strokes) # In a real example, this would be where you normalize the data.
    return processed_data

In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

def create_rnn_model(input_shape):
    """Creates a simple RNN model."""
    model = keras.Sequential([
        keras.layers.LSTM(128, input_shape=input_shape, return_sequences=True),
        keras.layers.LSTM(64),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(27, activation='softmax')  # 26 letters + space
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Example usage (assuming input_shape):
# model = create_rnn_model(input_shape=(None, 3)) # (time steps, features)
# Note: Input shape will depend on how you convert the raw stroke data.

In [ ]:
def train_model(model, train_data, train_labels):
    """Trains the RNN model."""
    # Placeholder: Create and train on a small dataset
    # model.fit(train_data, train_labels, epochs=10)
    print("Training function placeholder.")
    pass